In [1]:
%load_ext autoreload
%autoreload 2


from final_project.smlm_3d.config.datasets import dataset_configs
from final_project.smlm_3d.data.datasets import GenericDataSet, ExperimentalDataSet


class StormDataset(ExperimentalDataSet):
    def __init__(self, config, lazy=False, transform_data=False, normalize_psf=True):
        super().__init__(config, filter_localisations=False, lazy=True, transform_data=transform_data, normalize_psf=normalize_psf)
        self.filter_emitters_proximity=False
        self.filter_emitters_edges=True
        print(self.img.shape)
        if not lazy:
            self.prepare_data()

cfg = dataset_configs['nuclear_pore']['a2']
storm_ds = StormDataset(cfg)


Reading img...


  2%|▏         | 150/6192 [00:00<00:04, 1493.58it/s]

Loading /home/miguel/Projects/uni/data/smlm_3d/20210930_Nuclear_pore/exp2/647_STORM/A2/4631144/../MMStack_Default.ome.tif
Loading /home/miguel/Projects/uni/data/smlm_3d/20210930_Nuclear_pore/exp2/647_STORM/A2/4631144/MMStack_Default_final_subset.csv
(15000, 128, 128)
9999 emitters before filtering
6192 emitters after borders


100%|██████████| 6192/6192 [00:04<00:00, 1525.97it/s]


In [4]:
storm_ds.data[0].shape

(6192, 32, 32, 1)